In [1]:
import numpy as np 
import pandas as pd
import matplotlib.pyplot as plt
import cv2
import scipy 
import os
import keras
import sklearn
%matplotlib inline

Using TensorFlow backend.


In [17]:
train=pd.read_csv("Cleaned_Train_Data.csv")
test=pd.read_csv("Cleaned_Test_Data.csv")

In [15]:
train.shape

(69897, 6)

In [18]:
test.shape

(14952, 5)

In [19]:
train_img_path='Train_Data/Train/'
test_img_path='Test_Data/Test/'

In [20]:
train_img_names=os.listdir(train_img_path)
test_img_names=os.listdir(test_img_path)

In [21]:
train.head()

,Brand,Category,Gender,Color,Image_Names,Sub_category
0,Roadster,Tshirts,Men,Black,0_Graphic.jpg,Graphic
1,Roadster,Tshirts,Men,Black,1_Graphic.jpg,Graphic
2,Roadster,Tshirts,Men,Black,2_Biker.jpg,Biker
3,Roadster,Tshirts,Men,Black,3_Graphic.jpg,Graphic
4,Roadster,Tshirts,Men,Black,4_Graphic.jpg,Graphic


In [22]:
test.head()

,Brand,Category,Gender,Color,Image_Names
0,Allen Solly,Tshirts,Men,Red,0.jpg
1,Celio,Tshirts,Men,Yellow,1.jpg
2,CULT FICTION,Tshirts,Men,Rust,2.jpg
3,Antigravity,Tshirts,Women,Blue,3.jpg
4,Being Human,Tshirts,Men,Charcoal,4.jpg


In [23]:
def read_img(img_path):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (64,64))
    return img

In [25]:
from tqdm import tqdm

In [26]:
train_img = []
for img in tqdm(train['Image_Names'].values):
    train_img.append(read_img(train_img_path + img))

100%|██████████| 69897/69897 [1:10:04<00:00, 16.62it/s]


In [27]:
X_train=np.array(train_img,np.float32)
X_train=X_train/255.0

In [28]:
labels= train['Sub_category'].tolist()
Y_train = {k:v+1 for v,k in enumerate(set(labels))}
y_train = [Y_train[k] for k in labels]
from keras.utils import to_categorical
y_train = to_categorical(y_train)

In [29]:
y_train.shape

(69897, 25)

## Test Data Preprocessing

In [30]:
test.head()

,Brand,Category,Gender,Color,Image_Names
0,Allen Solly,Tshirts,Men,Red,0.jpg
1,Celio,Tshirts,Men,Yellow,1.jpg
2,CULT FICTION,Tshirts,Men,Rust,2.jpg
3,Antigravity,Tshirts,Women,Blue,3.jpg
4,Being Human,Tshirts,Men,Charcoal,4.jpg


In [31]:
test_img = []
for img in tqdm(test['Image_Names'].values):
    test_img.append(read_img(test_img_path + img))

100%|██████████| 14952/14952 [14:20<00:00, 17.38it/s]


In [32]:
X_test=np.array(test_img,np.float32)/255.0

In [33]:
mean_img=np.mean(X_train,axis=0)
std_img=np.std(X_train,axis=0)
X_train_norm=(X_train-mean_img)/std_img
X_test_norm=(X_test-mean_img)/std_img

In [34]:
Y_train

{'Abstract': 24,
 'Biker': 19,
 'Camouflage': 11,
 'Checked': 22,
 'Colourblocked': 13,
 'Conversational': 23,
 'Floral': 7,
 'Geometric': 14,
 'Graphic': 17,
 'Horizontal Stripes': 8,
 'Humour and Comic': 15,
 'Music': 3,
 'People and Places': 12,
 'Polka Dots': 9,
 'Self Design': 6,
 'Solid': 5,
 'Sports': 4,
 'Sports and Team Jersey': 21,
 'Striped': 20,
 'Superhero': 18,
 'Tie and Dye': 2,
 'Tribal': 1,
 'Typography': 16,
 'Varsity': 10}

In [35]:
len(Y_train)

24

In [36]:
y_train.shape

(69897, 25)

In [42]:
y_train=np.array(pd.get_dummies(train['Sub_category']))

In [43]:

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Convolution2D, MaxPooling2D
from keras.callbacks import EarlyStopping

In [46]:
model = Sequential()
model.add(Convolution2D(32, (3,3), activation='relu', padding='same',input_shape = (64,64,3))) # if you resize the image above, change the shape
model.add(Convolution2D(32, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Convolution2D(64, (3,3), activation='relu', padding='same'))
model.add(Convolution2D(64, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Convolution2D(128, (3,3), activation='relu', padding='same'))
model.add(Convolution2D(128, (3,3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(loss = 'categorical_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead
Instructions for updating:
keep_dims is deprecated, use keepdims instead


In [47]:
early_stops = EarlyStopping(patience=3, monitor='val_acc')
model.fit(X_train_norm, y_train, batch_size=100, epochs=10, validation_split=0.3, callbacks=[early_stops])

In [ ]:
model.fit(X_train_norm, y_train, batch_size=100, epochs=10, validation_split=0.3, callbacks=[early_stops])

Train on 48927 samples, validate on 20970 samples
Epoch 1/10
48927/48927 [==============================] - 3203s 65ms/step - loss: 1.7800 - acc: 0.5064 - val_loss: 1.6607 - val_acc: 0.5267
Epoch 2/10
48927/48927 [==============================] - 3236s 66ms/step - loss: 1.4926 - acc: 0.5838 - val_loss: 1.5306 - val_acc: 0.5601
Epoch 3/10
28800/48927 [================>.............] - ETA: 18:26 - loss: 1.4115 - acc: 0.6028